# Analyse de sentiments

Implémentation d'un prédicteur naïf de la polarité d'une critique de film tirée de IMDB anglais. 
Le modèle est un modèle de régression logistique traditionnel mais implémenté avec *pytorch*.
Le jeu de données est à télécharger depuis le site suivant : http://ai.stanford.edu/~amaas/data/sentiment/

## Loading DataSet

In [1]:
import os
import os.path
import random
from collections import Counter


random.seed(1) #reproductibility

#loads data from disk
def load_dataset(dir_path,ref_label, dev_test=False):
    """
    Loads a dataset from a directory path and 
    returns a list of couples (Counter of Bow_freq,ref_label) one for each text
    """
    dpath    = os.path.abspath(dir_path) 
    if dev_test:
        devset = []
        testset = []
        for f in os.listdir(dpath)[250:]:
            filepath    = os.path.join(dpath,f)
            file_stream = open(filepath)
            text        = file_stream.read().split()
            file_stream.close()
            devset.append((Counter(text),ref_label))
        for f in os.listdir(dpath)[250:600]:
            filepath    = os.path.join(dpath,f)
            file_stream = open(filepath)
            text        = file_stream.read().split()
            file_stream.close()
            testset.append((Counter(text),ref_label))
        return devset, testset
    else:
        data_set = [] 
        for f in os.listdir(dpath):
            filepath    = os.path.join(dpath,f)
            file_stream = open(filepath)
            text        = file_stream.read().split()
            file_stream.close()
            data_set.append((Counter(text),ref_label))
        return data_set

### Loading train set

In [2]:
trainpos = load_dataset("/Users/lara/Documents/LI/M2/Analyse_syntaxique/aclImdb/train/pos",1) 
trainneg = load_dataset("/Users/lara/Documents/LI/M2/Analyse_syntaxique/aclImdb/train/neg",0)
training_dataset  = trainpos
training_dataset.extend(trainneg)
random.shuffle(training_dataset)

### Loading test set

In [2]:
devpos, testpos = load_dataset("/Users/lara/Documents/LI/M2/Analyse_syntaxique/aclImdb/test/pos",1, dev_test=True)
devneg, testneg = load_dataset("/Users/lara/Documents/LI/M2/Analyse_syntaxique/aclImdb/test/neg",0, dev_test=True)
testing_dataset = testpos
dev_dataset = devpos
testing_dataset.extend(testneg)
dev_dataset.extend(devneg)
random.shuffle(testing_dataset)
random.shuffle(dev_dataset)

## Codage

In [4]:
#creates a dict that maps each known word string to a unique integer
def make_w2idx(dataset):
    wordset = set([])
    for X,Y in dataset:
        wordset.update([word for word in X])
    return dict(zip(wordset,range(len(wordset))))   

def vectorize_text(counter,w2idx):
    xvec = torch.zeros(len(w2idx))
    for word in counter:
        if word in w2idx:       #manages unk words (ignored)
            xvec[w2idx[word]] = counter[word] 
    return xvec.squeeze()

def vectorize_target(ylabel):
     return torch.tensor(float(ylabel))

## Modèle

La classe SentimentAnalyzer hérite de la classe torch.nn.Module. C'est une classe de base pour tous les modules du réseau de neurones.
La fonction reset_structure prend en arguments le réseau de neurones d'analyse de sentiments, la taille du vocabulaire (ce qui correspond à la taille de l'input) et le nombre de classes (ce qui correspond à la taille de l'output) et applique une transformation linéaire sur les données d'entrée (et donc sur la couche d'entrée) du réseau de neurones en renvoyant la matrice de poids W.
La fonction forward applique la propagation avant du réseau de neurones avec la fonction d'activation sigmoid qu'il cherche dans la bibliothèque fonctionnelle des réseaux de neurones de pytorch.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

class SentimentAnalyzer(nn.Module): 
    
        def __init__(self):
            
            super(SentimentAnalyzer, self).__init__()
            self.reset_structure(1,1)
            
        def reset_structure(self,vocab_size, num_labels):
            
            self.W = nn.Linear(vocab_size, num_labels)
            
        def forward(self, text_vec):
            
            return F.sigmoid(self.W(text_vec)) #sigmoid is the logistic activation
        
        def train(self,train_set, dev_set, learning_rate,epochs):
            
            self.w2idx = make_w2idx(train_set)
            self.reset_structure(len(self.w2idx),1)
            
            #remind that minimizing Binary Cross Entropy <=> minimizing NLL
            loss_func  = nn.BCELoss() 
            optimizer  = optim.SGD(self.parameters(), lr=learning_rate)
            
            for epoch in range(epochs):
                global_logloss = 0.0
                for X, Y in train_set: 
                    self.zero_grad() # sets gradients of all model parameters to 0
                    xvec            = vectorize_text(X,self.w2idx)
                    yvec            = vectorize_target(Y)
                    prob            = self(xvec).squeeze()
                    # appel implicite de la fonction forward --> forward propagation sur vecteur x
                    # .squeeze() retourne un tenseur avec toutes les dimensions d'entrée de taille 1 enlevées.
                    loss            = loss_func(prob,yvec)
                    loss.backward()
                    optimizer.step()
                    global_logloss += loss.item()
                for X, Y in dev_set: 
                    # On entraine le réseau sur l'ensemble de developpement
                    # mais on ne le modifie pas ( pas de backward propagation )
                    self.zero_grad() # sets gradients of all model parameters to 0
                    xvec            = vectorize_text(X,self.w2idx)
                    yvec            = vectorize_target(Y)
                    prob            = self(xvec).squeeze()
                    loss            = loss_func(prob,yvec)
                    global_logloss += loss.item()

                print("Epoch %d, mean cross entropy = %f"%(epoch, global_logloss/len(dev_set)))

        def run_test(self,test_set):
            # renvoie un score d'accurracy de classification sur un jeu de test.
            with torch.no_grad():
                acc = 0.0
                total = 0.0
                for X, Y in test_set:
                    total += 1
                    xvec = vectorize_text(X,self.w2idx)
                    yvec = vectorize_target(Y)
                    yhat = self(xvec).squeeze()
                    if (yvec ==1 and yhat >= 0.5) or (yvec == 0 and yhat < 0.5):
                        acc += 1
            return acc * 100 / total

ImportError: numpy.core.multiarray failed to import

## Inférences

In [ ]:
sent = SentimentAnalyzer()

## Grid Search

In [3]:
import csv
i = 0 
with open('grid_search.csv', 'w') as csvfile:
    grid_search = csv.writer(csvfile, delimiter=";")
    grid_search.writerow(['GD', 'epochs', 'lr', 'accuracy', 'mean cross entropy at last epoch'])
    for epoch in [100, 500, 1000]:
        for lr in [0.001, 0.01, 0.1, 1, 10]:
            mce = sent.train(training_dataset,dev_dataset, lr,epoch)
            acc = sent.run_test(testing_dataset)
            grid_search.writerow([i, epoch, lr, acc, mce])
            


NameError: name 'sent' is not defined